In [ ]:
# Importer les bibliothèques
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from scipy.sparse import hstack, csr_matrix
import joblib
import nltk
from nltk.corpus import stopwords
from flask import Flask, request, jsonify
from flask_cors import CORS

# -----------------------------
# 0️⃣ Télécharger les stopwords français
# -----------------------------
nltk.download('stopwords')
french_stopwords = stopwords.words('french')

# -----------------------------
# 1️⃣ Charger le dataset multilingue (SMS Spam Multilingual) et filtrer le français
# -----------------------------
dataset = load_dataset("dbarbedillo/SMS_Spam_Multilingual_Collection_Dataset")

# Récupérer uniquement les messages français
texts_fr = [x["text_fr"] for x in dataset["train"]]  # colonne pour français
labels = [1 if x["labels"] == "spam" else 0 for x in dataset["train"]]  # colonne "labels"

print("Nombre de messages :", len(texts_fr))
print("Labels disponibles :", set(labels))
print("Exemple message :", texts_fr[0], "| Label :", labels[0])

# -----------------------------
# 2️⃣ Split train/test
# -----------------------------
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts_fr, labels, test_size=0.2, random_state=42, stratify=labels
)

# -----------------------------
# 3️⃣ TF-IDF (avec stopwords français)
# -----------------------------
vectorizer = TfidfVectorizer(stop_words=french_stopwords, max_df=0.9, max_features=2000)
X_train_tfidf = vectorizer.fit_transform(X_train_texts)
X_test_tfidf = vectorizer.transform(X_test_texts)

# -----------------------------
# 4️⃣ Features manuelles supplémentaires
# -----------------------------
def add_manual_features(texts):
    """
    Features simples pour le spam :
    - nb de chiffres
    - nb de symboles spéciaux !,$,€
    - longueur du message
    """
    digits = [sum(c.isdigit() for c in t) for t in texts]
    symbols = [sum(c in "!$€" for c in t) for t in texts]
    length = [len(t) for t in texts]
    return np.array([digits, symbols, length]).T

X_train_manual = csr_matrix(add_manual_features(X_train_texts))
X_test_manual = csr_matrix(add_manual_features(X_test_texts))

# -----------------------------
# 5️⃣ Combiner TF-IDF + features manuelles
# -----------------------------
X_train_final = hstack([X_train_tfidf, X_train_manual])
X_test_final = hstack([X_test_tfidf, X_test_manual])

print("Données combinées TF-IDF + features manuelles ✅")

# -----------------------------
# 6️⃣ Entraîner Random Forest
# -----------------------------
rf_model = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight='balanced'
)
rf_model.fit(X_train_final, y_train)
print("Modèle Random Forest entraîné ✅")

# -----------------------------
# 7️⃣ Évaluer le modèle
# -----------------------------
preds = rf_model.predict(X_test_final)
print("Accuracy :", accuracy_score(y_test, preds))
print("\nClassification Report :\n", classification_report(y_test, preds, target_names=["HAM", "SPAM"]))

# -----------------------------
# 8️⃣ Sauvegarder le modèle et le vectorizer
# -----------------------------
joblib.dump(rf_model, "spam_rf_model_fr.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer_fr.pkl")
print("Modèle et vectorizer sauvegardés ✅")

# -----------------------------
# 9️⃣ Fonction pour prédire un nouveau message
# -----------------------------
def predict_message(message):
    tfidf_vec = vectorizer.transform([message])
    manual_vec = csr_matrix(add_manual_features([message]))
    combined_vec = hstack([tfidf_vec, manual_vec])
    pred = rf_model.predict(combined_vec)[0]
    conf = rf_model.predict_proba(combined_vec)[0][pred] * 100
    return {"prediction": "SPAM" if pred==1 else "HAM", "confidence": round(conf,2)}

# -----------------------------
# 🔟 Tests
# -----------------------------
message_test_spam = "Je n'y connaissais rien en informatique ou en finance. En suivant simplement les instructions pendant 10 minutes par jour, j'ai généré 4 500 € dès la première semaine. Ma vie a totalement changé, merci !"
message_test_ham = "Salut, on se retrouve demain pour le cours de maths ?"

result_spam = predict_message(message_test_spam)
result_ham = predict_message(message_test_ham)

print("\nTest SPAM :", message_test_spam)
print("Prédiction :", result_spam["prediction"], "| Confiance :", result_spam["confidence"], "%")

print("\nTest HAM :", message_test_ham)
print("Prédiction :", result_ham["prediction"], "| Confiance :", result_ham["confidence"], "%")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Nombre de messages : 5572
Labels disponibles : {0, 1}
Exemple message : Allez jusqu'à Jurong point, fou.. Disponible seulement dans bugis n grand monde la e buffet... Cine il y a eu plus... | Label : 0
Données combinées TF-IDF + features manuelles ✅
Modèle Random Forest entraîné ✅
Accuracy : 0.9883408071748879

Classification Report :
               precision    recall  f1-score   support

         HAM       0.99      1.00      0.99       966
        SPAM       1.00      0.91      0.95       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

Modèle et vectorizer sauvegardés ✅

Test SPAM : Je n'y connaissais rien en informatique ou en finance. En suivant simplement les instructions pendant 10 minutes par jour, j'ai généré 4 500 € dès la première semaine. Ma vie a totalement changé, merci !
Prédiction : SPAM | Confiance : 67.67 %

Test HAM : Salut, on se retrouve demain po